# Movielens - Created by Jesús García García - Masterś Degree in Big Data Analytics UEM


In [1]:
# Source https://spark.apache.org/docs/2.2.0/mllib-collaborative-filtering.html

In [2]:
import warnings
import time
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating


# Delete execution warnings
warnings.filterwarnings('ignore')
# Stop SparkContext previously created when we put "pyspark" command on Terminal
#sc.stop()

# set up environment
#conf=SparkConf()
# conf.set("spark.executor.memory", "3g")
#conf.set("spark.cores.max", "2")
#conf.setAppName("MovieLensJGG")
#conf.setMaster('localhost')
#sc = SparkContext('local', conf=conf)


# Load and parse the data
data = sc.textFile("file:///home/cloudera/Downloads/ratings1k.csv")
ratings = data.map(lambda l: l.split(','))\
    .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
# Train 80%, Test 20%    
trainData, testData = ratings.randomSplit([0.8,0.2],seed=42)

In [3]:
# Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = ALS.train(trainData, rank, numIterations)

# Evaluate the model on training data
now = time.time()
testdata = testData.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
elapsed = (time.time() - now)
print "Average time for evaluating the model on training data: {:.2f}ms".format(elapsed)
print("Mean Squared Error = " + str(MSE))
print(predictions)
print(ratesAndPreds)

Average time for evaluating the model on training data: 10.64ms
Mean Squared Error = 1.14843817893
PythonRDD[233] at RDD at PythonRDD.scala:43
PythonRDD[234] at RDD at PythonRDD.scala:43


In [4]:
# Show 'userId','movieId','prediction' zipped by index
predictions.zipWithIndex().take(10)

[(((988, 1084), 4.255391396106692), 0),
 (((123, 1084), 3.970427485966827), 1),
 (((312, 1084), 3.661339484090824), 2),
 (((702, 1084), 3.770791318492449), 3),
 (((727, 1084), 3.593052318835719), 4),
 (((173, 1084), 4.6670848572988906), 5),
 (((59, 1084), 3.1516930325966452), 6),
 (((673, 1084), 3.955842458166887), 7),
 (((424, 1084), 4.627669131068226), 8),
 (((366, 1084), 3.950863921901613), 9)]

In [5]:
# Show 'userId','movieId','prediction' in descending order by prediction
predictions.takeOrdered(10, key = lambda x: -x[1])

[((200, 2041), 10.47502047650468),
 ((861, 449), 9.15823021388792),
 ((310, 2236), 8.409670434047163),
 ((812, 2975), 8.397771447515511),
 ((337, 2265), 8.20397173807137),
 ((573, 190), 8.068599169156053),
 ((846, 714), 7.955456895494173),
 ((191, 193), 7.905061375553057),
 ((455, 105), 7.904580206351197),
 ((279, 3742), 7.761459254303782)]

In [6]:
# Show 'userId','movieId','prediction' in ascending order by prediction
predictions.takeOrdered(10, key = lambda x: x[1])

[((87, 390), -6.293869029812393),
 ((540, 2338), -5.583789598553501),
 ((836, 1750), -4.610322247242783),
 ((46, 1981), -4.304710690287035),
 ((836, 1772), -4.071023590723346),
 ((602, 3340), -3.7610045627854314),
 ((532, 3287), -3.452284422672739),
 ((673, 193), -3.151288697306291),
 ((800, 2095), -2.908144127581073),
 ((732, 2027), -2.8381414235429285)]

In [7]:
# Show 'userId','movieId','rating','prediction' zipped by index
ratesAndPreds.zipWithIndex().take(10)

[(((822, 1620), (4.0, 3.522175333778284)), 0),
 (((302, 112), (2.0, 3.508042099046171)), 1),
 (((528, 1136), (4.0, 3.515727606467057)), 2),
 (((855, 849), (2.0, 1.8538018953810358)), 3),
 (((238, 3504), (5.0, 4.130894994959826)), 4),
 (((449, 2797), (4.0, 4.605564419977343)), 5),
 (((424, 2148), (3.0, 3.0164966591088778)), 6),
 (((451, 3169), (3.0, 3.4964842676166237)), 7),
 (((173, 3547), (4.0, 4.140044827711459)), 8),
 (((620, 380), (4.0, 3.4627204555540967)), 9)]

In [8]:
# Show 'userId','movieId','rating','prediction' in ascending order by rating
ratesAndPreds.takeOrdered(10,key =lambda x: x[1])

[((87, 390), (1.0, -6.293869029812393)),
 ((540, 2338), (1.0, -5.583789598553501)),
 ((836, 1750), (1.0, -4.610322247242783)),
 ((836, 1772), (1.0, -4.071023590723346)),
 ((673, 193), (1.0, -3.151288697306291)),
 ((732, 2027), (1.0, -2.8381414235429285)),
 ((745, 3886), (1.0, -2.2215861482054153)),
 ((335, 44), (1.0, -1.775156133049732)),
 ((602, 3937), (1.0, -1.6015972075609235)),
 ((60, 1731), (1.0, -1.454655725150434))]

In [9]:
# Show 'userId','movieId','rating','prediction' in descending order by rating
# It's needed to convert RDD to Pandas Dataframe
import pandas as pd
dfaux = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions).toDF()
df = dfaux.toPandas()
df.columns = ['userId_movieId','rating_prediction']
df.sort(['rating_prediction'], ascending=False)

,userId_movieId,rating_prediction
17781,"(200, 2041)","(5.0, 10.4750204765)"
14497,"(861, 449)","(5.0, 9.15823021389)"
11579,"(310, 2236)","(5.0, 8.40967043405)"
23969,"(573, 190)","(5.0, 8.06859916916)"
14330,"(846, 714)","(5.0, 7.95545689549)"
29773,"(673, 1046)","(5.0, 7.75040986584)"
3426,"(786, 1702)","(5.0, 7.42168585555)"
21,"(281, 1699)","(5.0, 7.3313443003)"
28668,"(357, 1258)","(5.0, 7.32326744128)"
23318,"(315, 2414)","(5.0, 7.31420138582)"


In [11]:
# Save and load model
model.save(sc, "file:///home/cloudera/Downloads/ALSJGGModel")
sameModel = MatrixFactorizationModel.load(sc, "file:///home/cloudera/Downloads/ALSJGGModel")
sameModel